In [233]:
import pandas as pd
#!pip install twokenize
#import twokenize

In [234]:
df_wikipedia=pd.read_csv("../data/train_wikipedia.csv")
#df2 = {'id': 31234234, 'comment_text': 'I love Pizza 🍕! Shall we book a cab 🚕 to gizza?', 'toxic':0,
#'severe_toxic':0,'obscene':1,'threat':1,'insult':1,'identity_hate':1}
#df_wikipedia = df_wikipedia.append(df2, ignore_index = True)
df_wikipedia.tail()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0
223548,fffb5451268fb5ba,""" \n\n == Unicorn lair discovery == \n\n Suppo...",0,0,0,0,0,0


# Remove double quotations "

In [235]:
def remove_doublequotation(panda_text_column):
    return panda_text_column.apply(lambda x: x.replace('"', ''))
df_wikipedia['comment_text'] = remove_doublequotation(df_wikipedia['comment_text'])
df_wikipedia.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Handle Non-Text 

In [138]:
def handle_nontext(panda_column):
    return panda_column.apply(lambda x: x.encode("utf-8"))
#df_wikipedia["comment_text"] = handle_nontext(df_wikipedia["comment_text"] )
#df_wikipedia.tail()

# then bytes make it diffcult to deal with data

## Handling apostrophes (manually)

In [236]:
Apostrophes_expansion = {
"i'm": "i am","I'm": "I am", "I'M": "I AM", 
"i'd": "i would", "I'd": "I would","I'D": "I WOULD",
"you're": "you are", "You're": "You are", "YOU'RE": "YOU ARE",
"he's": "he is","He's": "He is","HE'S": "HE IS",
"she's": "she is","She's": "She is","SHE'S": "SHE IS",
"it's":"it is", "It's":"It is", "IT'S":"IT IS",
"that's": "that is", "That's": "That is","THAT'S": "THAT IS",
"they're": "they are","They're": "They are","THEY'RE": "THEY ARE",
"we're": "we are","We're": "We are","WE'RE": "WE ARE",
"i've": "i have", "I've": "I have", "I'VE": "I HAVE",
"you've": "you have","You've": "You have","YOU'VE": "YOU HAVE",
"we've": "we have","We've": "We have","WE'VE": "WE HAVE",
"ain't": "are not","Ain't": "Are not","AIN'T": "ARE NOT",
"aren't": "are not", "Aren't": "Are not","AREN'T": "ARE NOT",
"isn't": "is not", "Isn't": "Is not", "ISN'T": "IS NOT",
"don't": "do not", "Don't": "Do not","DON'T": "DO NOT",
"doesn't": "does not", "Doesn't": "Does not", "DOESN'T": "DOES NOT",
"won't": "will not","Won't": "Will not", "WON'T": "WILL NOT",
"can't": "cannot","Can't": "Cannot", "CAN'T": "CANNOT",
"shouldn't": "should not","Shouldn't": "Should not","SHOULDN'T": "SHOULD NOT",
"wouldn't": "would not", "Wouldn't": "Would not", "WOULDN'T": "WOULD NOT",
"didn't": "did not", "Didn't": "Did not", "DIDN'T": "DID NOT",
"weren't":"were not","Weren't":"Were not", "WEREN'T":"WERE NOT", 
"wasn't":"was not","Wasn't":"Was not", "WASN'T":"WAS NOT", 
"there's": "there is", "There's": "There is", "THERE'S": "THERE IS",
"here's": "here is", "Here's": "Here is", "HERE'S": "HERE IS",
"let's": 'let us', "Let's": "Let us", "LET'S": "LET US"
} 

def interpolate_apostrophes_string(string):
    x = string.split(' ')
    return " ".join([Apostrophes_expansion[word] if word in Apostrophes_expansion else word for word in x])
def interpolate_apostrophes_column(panda_text_column):
    return panda_text_column.apply(lambda x: interpolate_apostrophes_string(x))
df_wikipedia['comment_text'] = interpolate_apostrophes_column(df_wikipedia.comment_text)
df_wikipedia.to_csv('../data/test_apostrophes.csv')

## Nonstandard spellings

In [237]:
#!pip install textblob
def correct_spelling(panda_text_column):
    from textblob import TextBlob
    return panda_text_column.apply(lambda x: TextBlob(x).correct())

df_wikipedia['comment_text'] = correct_spelling(df_wikipedia['comment_text'])

In [140]:
df_wikipedia['comment_text'].head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I am ...
2    Hey man, I am really not trying to edit war. I...
3    \nMore\nI cannot make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

## Removal of URLs

In [144]:
!pip install regex
def process_URLs(string): 
    '''
    replace all URLs in the tweet text
    '''
    UrlStart1 = regex_or('https?://', r'www\.')
    CommonTLDs = regex_or( 'com','co\\.uk','org','net','info','ca','biz',
                                'info','edu','in','au')
    UrlStart2 = r'[a-z0-9\.-]+?' + r'\.' + CommonTLDs + pos_lookahead(r'[/ \W\b]')
    
    # * not + for case of: "go to bla.com." -- don't want period UrlBody = r'[^ \t\r\n<>]*?'
    UrlExtraCrapBeforeEnd = '%s+?' % regex_or(PunctChars, Entity) 
    UrlEnd = regex_or( r'\.\.+', r'[<>]', r'\s', '$')
    Url = (optional(r'\b') +
                regex_or(UrlStart1, UrlStart2) +
                UrlBody +
    pos_lookahead( optional(UrlExtraCrapBeforeEnd) + UrlEnd))
    
    Url_RE = re.compile("(%s)" % Url, re.U|re.I)
    string = re.sub(Url_RE, " constant_url ", string)
    
    # fix to handle unicodes in URL
    URL_regex2 = r'\b(htt)[p\:\/]*([\\x\\u][a-z0-9]*)*' 
    string = re.sub(URL_regex2, " constant_url ", string) 
    return string

process_URLs("strinHey man, I am reallg www.facebook.com")

NameError: name 'regex_or' is not defined

## Translate emojis into words

In [232]:
#!pip install demoji
#import demoji
demoji.download_codes()
# get dictionary of emojis
def get_emojis(panda_text_column):
    dict = {}
    for row in panda_text_column:
        dict.update(demoji.findall(row))
    return dict
x = get_emojis(df_wikipedia['comment_text'][:100])
print(x)
#print(demoji.findall(tweet))
#def get_emojis(panda_text_column):
#    return panda_text_column.apply(lambda x: demoji.findall(x))
#x = get_emojis(df_wikipedia['comment_text'][:1000] )


def remove_emojis(df, column):
    df_copy = df.copy()
    for i,text in enumerate(df_copy[column]):
        dem = demoji.findall(text)
        if dem:
            for item in dem.keys():
                df_copy[column][i] = text.replace(item, '')
    return df_copy

remove_emojis(df_wikipedia, 'comment_text'])
for i,text in enumerate(df_wikipedia['comment_text']):
    dem = demoji.findall(text)
    if dem:
        for item in dem.keys():
            df_wikipedia['comment_text'][i] = text.replace(item, '')

/var/folders/wp/ylcd0z3n66jgx_wrcv4p13v80000gn/T/ipykernel_75753/621620625.py:3: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


{}


/var/folders/wp/ylcd0z3n66jgx_wrcv4p13v80000gn/T/ipykernel_75753/621620625.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wikipedia['comment_text'][i] = text.replace(item, '')


KeyboardInterrupt: 

In [220]:
tweet = "I love Pizza 🍕! Shall we book a cab 🚕 to gizza?"
dem = demoji.findall(tweet)
for item in dem.keys():
    tweet = tweet.replace(item, '')
tweet

'I love Pizza ! Shall we book a cab  to gizza?'

In [193]:
from collections import ChainMap
d6 = ChainMap(x)
d6

ChainMap(0      {}
1      {}
2      {}
3      {}
4      {}
       ..
995    {}
996    {}
997    {}
998    {}
999    {}
Name: comment_text, Length: 1000, dtype: object)

## Save

In [ ]:
df_wikipedia.to_csv('train_wikipedia_preprocessed.csv', index=False)